# Stats for eHRAF Scraper
The current stats are merely to reorganize the dataframes and find association rules (when one OCM appears this other OCM is likely to appear)
More work will be done

### Optional dataframe manipulation

In [45]:
import pandas as pd                 # dataframe storing
import re                           # regex for searching through strings



# put in file here
# df = pd.read_excel('Data/text~Apple|PSF_web_data.xlsx')
df = pd.read_excel('../Data/subjects~(%22sickness%22+OR+%22preventive+medicine%22+OR+%22bodily+injuries%22+OR+%22sorcery%22)|PSF_web_data.xlsx')

# Turn the string of column OCM back into a list 
df['OCM'] = df.OCM.apply(lambda x: re.sub(" ",'',x))
df['OCM'] = df.OCM.apply(lambda x: x[1:-1].split(','))

# did it work? did it output a single OCM string?
df['OCM'][0][0]

'755'

In [46]:
culture_set = set(df["Culture"])
culture_dict = {}
for cult_i in culture_set:
    row_count = len(df.loc[df["Culture"]==cult_i])
    culture_dict[cult_i] = row_count

In [47]:
# drop all rows who have a blank passage or 
print(f'Before: {len(df)}')
df = df.dropna(subset="Passage")
print(f'After: {len(df)}')

Before: 42555
After: 42458


In [48]:
# Because eHRAF seems to aquire ALL OCMs related to your input, not just your specified, 
# select only the OCMs we originally wished to search for (750-754)
lst = ["750","751","752","753","754"]
msk = df['OCM'].apply(lambda x: not set(x).isdisjoint(lst))
df = df.loc[msk]
len(df)

14866

In [49]:
# (OPTIONAL)
# for searching by OCM in the list
lst = ["1787"]
msk = df['OCM'].apply(lambda x: not set(x).isdisjoint(lst))
out = df.loc[msk]
out

,Region,SubRegion,Culture,DocTitle,Year,OCM,OWC,Passage,run_Info


In [42]:
shortPass_list = []
for i in df['Passage']:
    if len(i)<=10:
        shortPass_list.append(i)
print(f'Number of passages with text with 10 of fewer characters: {len(shortPass_list)}')

Number of passages with text with 10 of fewer characters: 45


## OCM Code Counting
- Count every OCM within each culture and count them. Do not count OCM's specified by the search (like if searched for 750-755, do not count these). 
- REMOVE all passages which are blank since we can't very well do lexical searches on them

In [50]:
# Make a dataset in which each OCM have its own row by exploding (you can reset the index with .reset_index(drop=True))
df_OCM = df.explode(column='OCM').reset_index(drop=True)
# Find OCM's that do not fit the normal 100-900 OCM scheme
# NOTE 0 means the material is not relevant, I am unsure, however, why this sometimes appears with other OCM's in the same passage
# NOTE I believe 5310 and 5311 are different specifications of 531 while 1710 might be a more specific (and singlular) subset of 171? I do not believe the same for 77 and 1787
list_OCM = df_OCM['OCM'].value_counts().index.tolist()
small_OCM = [x for x in list_OCM if len(x) <3 or len(x) > 3]
small_OCM

['0', '5311', '5310']

In [51]:
# remove and shave OCM codes
# add to the list for codes which should be removed
remove_list = ['1787','77']
for i in remove_list:
    df_OCM = df_OCM[df_OCM["OCM"].str.contains(i) == False]
# "Shave" the OCM codes that seem to have a parent (5310 and 5311 become 531).
df_OCM['OCM'] = df_OCM.OCM.apply(lambda x: x[0:3] if len(x) >= 3 else x)

In [52]:
# Then turn the OCM's back to an integer (for removals)
df_OCM['OCM'] = df_OCM.OCM.apply(lambda x: int(x))

df_sub_ex = df_OCM.loc[(df_OCM["OCM"]<750) | (df_OCM["OCM"]>754)]

# Create a dataframe for OCM counts and frequencies
df_OCM = pd.DataFrame(columns=["Culture","OCM","Frequency","Proportion_of_Passages"])
for key, val in culture_dict.items():
    value_count = df_sub_ex.loc[df_sub_ex["Culture"]==key]["OCM"].value_counts()
    cult_count = [key] * len(value_count)
    df_OCM_Concat = pd.DataFrame({"Culture":cult_count,"OCM":value_count.index, "Frequency":value_count.values, "Proportion_of_Passages":value_count.values/val})
    df_OCM = pd.concat([df_OCM, df_OCM_Concat], ignore_index=True)
df_OCM

,Culture,OCM,Frequency,Proportion_of_Passages
0,Wolof,565,15,0.123967
1,Wolof,827,9,0.074380
2,Wolof,152,7,0.057851
3,Wolof,586,7,0.057851
4,Wolof,789,5,0.041322
...,...,...,...,...
6304,Amhara,743,1,0.001603
6305,Amhara,688,1,0.001603
6306,Amhara,557,1,0.001603
6307,Amhara,578,1,0.001603


In [53]:
# Save the file
df_OCM.to_csv("Culture_Frequency.csv", index=False)

## Running machine learning on OCM

In [5]:
# Make a dataset in which each OCM have its own row by exploding (you can reset the index with .reset_index(drop=True))
df_OCM = df.explode(column='OCM').reset_index(drop=True)
df_OCM

,Region,SubRegion,Culture,DocTitle,Year,OCM,OWC,Passage,run_Info
0,Africa,Central Africa,Azande,An account of the Zande,1926,137,fo07,The bagara looks like a miniature custard appl...,User: No Name Specified
1,Africa,Central Africa,Azande,An account of the Zande,1926,222,fo07,The bagara looks like a miniature custard appl...,User: No Name Specified
2,Africa,Central Africa,Azande,An account of the Zande,1926,252,fo07,The bagara looks like a miniature custard appl...,User: No Name Specified
3,Africa,Central Africa,Azande,An account of the Zande,1926,137,fo07,The bagara looks like a miniature custard appl...,Run Time: 10:43:01
4,Africa,Central Africa,Azande,An account of the Zande,1926,222,fo07,The bagara looks like a miniature custard appl...,Run Time: 10:43:01
...,...,...,...,...,...,...,...,...,...
1063,South-America,Southern South America,Mataco,The Mataco Indians and their language,1897,192,si07,One notices here the constant change of u to a...,NaN
1064,South-America,Southern South America,Mataco,The Mataco Indians and their language,1897,196,si07,One notices here the constant change of u to a...,NaN
1065,South-America,Southern South America,Ona,Analytical and critical bibliography of the tr...,1917,104,sh04,"4. Throat. Sk, j[unknown] e[unknown] ka[unknow...",NaN
1066,South-America,Southern South America,Ona,Analytical and critical bibliography of the tr...,1917,192,sh04,"4. Throat. Sk, j[unknown] e[unknown] ka[unknow...",NaN


In [6]:
from mlxtend.preprocessing import TransactionEncoder

# We will use the apriori module to generate a dataframe that
# we can use for association rule finding
from mlxtend.frequent_patterns import apriori

# We will use the association_rules module to generate
# our association rules from the apriori output data frame
from mlxtend.frequent_patterns import association_rules





In [7]:
# df['Index_Col'] = df.index
df_smaller = df_OCM[['Culture', 'OCM','Passage']]
df_smaller

,Culture,OCM,Passage
0,Azande,137,The bagara looks like a miniature custard appl...
1,Azande,222,The bagara looks like a miniature custard appl...
2,Azande,252,The bagara looks like a miniature custard appl...
3,Azande,137,The bagara looks like a miniature custard appl...
4,Azande,222,The bagara looks like a miniature custard appl...
...,...,...,...
1063,Mataco,192,One notices here the constant change of u to a...
1064,Mataco,196,One notices here the constant change of u to a...
1065,Ona,104,"4. Throat. Sk, j[unknown] e[unknown] ka[unknow..."
1066,Ona,192,"4. Throat. Sk, j[unknown] e[unknown] ka[unknow..."


In [8]:
df_group = df_smaller.groupby(by = ['Culture', 'Passage'])
df_group

In [9]:
def make_OCM_list(x):

    '''
    Will return a list of the unique items
    in a particular grouping when used with
    the agg method as its function
    '''

    return x.unique()

In [10]:
# Use the agg method and make_OCM_list
# to return a list of unique items for each ocm
df_unique = df_group.agg(make_OCM_list)

In [11]:
list_trans = list(df_unique['OCM'])
list_trans = list_trans[0:]
len(list_trans)

196

In [12]:
te = TransactionEncoder()
encoded_itemset = te.fit(list_trans).transform(list_trans)
print(encoded_itemset.shape) # show possible transcations and number of items
te.columns_



df_encoded = pd.DataFrame(encoded_itemset, columns = te.columns_)
df_encoded.head()

(196, 180)


,0,104,113,114,121,123,125,128,132,133,...,857,861,865,866,867,883,887,900,901,902
0,False,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [13]:
# Before we begin, let's do a small
# amount of cleanup.  Let's remove all
# columns (items) that have less than 1 characters since that is just blank space
# more data cleaning my be required as time continues in case errors become evident in the scraped dataset
OCM_items = list(filter(lambda x: len(x) < 1, te.columns_ ))
print("removed: ",  OCM_items)
df_encoded = df_encoded.drop(columns=OCM_items) #remove small strings as they seem not to be items
print('How many unique items are left?', len(df_encoded.columns))

removed:  []
How many unique items are left? 180


In [16]:
# Use apriori to create a dataframe with columns of support and itemset lists
# Note that if your items are large compared to your sample (you have few rows but many columns) I reccommend using 
# a higher min_support as many more combinations may have spuriously higher support. Also, you can crash the program if too many are selected
df_support = apriori(df_encoded, min_support=0.01, use_colnames=True)
df_support.sort_values('support', inplace=True, ascending = False)
df_support

,support,itemsets
9,0.132653,(137)
0,0.107143,(0)
78,0.107143,(824)
27,0.107143,(262)
24,0.102041,(245)
...,...,...
79,0.010204,(825)
77,0.010204,(804)
71,0.010204,(778)
69,0.010204,(773)


## Use association_rules to find the rules

Using the dataframe generated by `apriori`, find the association rules with the greatest lift.  See the [association_rules documentation](https://rasbt.github.io/mlxtend/api_modules/mlxtend.frequent_patterns/association_rules/) for how to do this.

Sort the resulting DataFrame by lift in descending order.  A lift > 1 indicates that the items are often purchased together and that buying X will increase the purchase of Y.  A lift of < 1 indicates the items are often substituted.  That is X is substituted for Y so X and Y don't appear together often.

Examine the resulting DataFrame.  For the association rule X -> Y, X is the column `antecedents` and Y is the column `consequents`.  If sorted you can see the metrics for each rule based upon the lift.

In [17]:
# Find the association rules
rules = association_rules(df_support, metric = 'lift', min_threshold=1.0)
# lift >1 more likely than chance X means you see Y
# lift = 1 as often as chance
# lift <1 (substitution) less likely than chance X means you see Y


In [18]:
# Sort the rules by lift
# and examine the output
# to find what rules were
# discovered
rules.sort_values('lift', ascending=False, inplace =True)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
112,"(291, 223, 231)",(132),0.010204,0.010204,0.010204,1.000000,98.000000,0.010100,inf
103,"(223, 132)","(231, 137)",0.010204,0.010204,0.010204,1.000000,98.000000,0.010100,inf
101,"(231, 132, 137)",(223),0.010204,0.010204,0.010204,1.000000,98.000000,0.010100,inf
246,"(231, 137)","(291, 132)",0.010204,0.010204,0.010204,1.000000,98.000000,0.010100,inf
99,"(223, 231, 137)",(132),0.010204,0.010204,0.010204,1.000000,98.000000,0.010100,inf
...,...,...,...,...,...,...,...,...,...
97,(262),(177),0.107143,0.071429,0.010204,0.095238,1.333333,0.002551,1.026316
143,(252),(245),0.076531,0.102041,0.010204,0.133333,1.306667,0.002395,1.036107
142,(245),(252),0.102041,0.076531,0.010204,0.100000,1.306667,0.002395,1.026077
264,(252),(137),0.076531,0.132653,0.010204,0.133333,1.005128,0.000052,1.000785


In [19]:
# look for OCM codes within the list
lst = frozenset(["241","226"])
msk = rules['antecedents'].apply(lambda x: not set(x).isdisjoint(lst))
out = rules.loc[msk]
out

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
69,(241),(242),0.045918,0.020408,0.010204,0.222222,10.888889,0.009267,1.259475
277,(241),(133),0.045918,0.020408,0.010204,0.222222,10.888889,0.009267,1.259475
24,(241),(137),0.045918,0.132653,0.015306,0.333333,2.512821,0.009215,1.301020
